In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/산대특 Project/프로젝트 준비/221121_최종 사진리뷰예측/Wlsakrakr.csv', index_col = 0)   # 찐막, 1730 결측치 없는 데이터

In [ ]:
# # row 생략 없이 출력
# pd.set_option('display.max_rows', None)
# # col 생략 없이 출력
# pd.set_option('display.max_columns', None)

In [ ]:
# 이미지, 소개글, 댓글만 각각 지정.
df_image = df.iloc[:,11:-1]        # 이미지 '%' 들어있음
df_text = df['소개글']
df_review = df['댓글']
df_score = df['평점']
df_price = df['가격']
df_cont = df.iloc[:,2:7]   # 통제변수들 가격, 최대인원수, 침실 침대 개수, 욕실개수
df_cont

,가격,최대_인원수,침실개수,침대개수,욕실개수
0,98940,4,1.0,1.0,1.0
1,59341,4,1.0,1.0,1.0
2,219562,5,2.0,1.0,1.0
3,84447,2,1.0,1.0,1.0
4,64000,2,1.0,1.0,1.0
...,...,...,...,...,...
1725,52951,2,0.0,1.0,1.0
1726,55918,4,1.0,2.0,1.0
1727,74177,2,1.0,1.0,1.0
1728,77030,4,1.0,1.0,1.0


In [ ]:
#이미지 데이터 전처리
import numpy as np
image = np.array(df_image)

# '%' 없애주기
for i in range(len(image)):
  for j in range(4):
    image[i][j] = image[i][j][:-1]

df_image = image
type(df_image)
df_image

array([['28.3', '16.3', '21.0', '34.6'],
       ['47.4', '7.8', '24.7', '20.2'],
       ['9.6', '33.2', '22.2', '34.8'],
       ...,
       ['15.6', '15.0', '42.7', '26.7'],
       ['47.4', '15.6', '6.4', '30.7'],
       ['28.3', '7.1', '48.8', '16.0']], dtype=object)

In [ ]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df.isna().sum()

숙소_형태      0
위치         0
가격         0
최대_인원수     0
침실개수       0
침대개수       0
욕실개수       0
소개글        0
댓글         0
댓글개수       0
평점         0
Casual     0
Classic    0
Modern     0
Natural    0
max_pre    0
dtype: int64

In [ ]:
# 각 스타일에 해당하는 10개 단어 (명사형으로 변형) -  텍스트
Casual = ['신선', '경쾌', '동적', '활동', '즐거운', '발랄', '재미', '친근', '따뜻', '산뜻']
Classic = ['고급', '우아', '고풍', '화려', '호화', '장식', '고전', '중후', '전통', '무거운']
Modern = ['단순', '현대', '깔끔', '깨끗', '간결', '간단', '심플', '기능', '도시', '합리', '절제', '세련']
Natural = ['자연', '여유', '온화', '부드러운', '포근', '전원', '편안', '솔직', '아늑', '자연의']

from konlpy.tag import Okt
okt = Okt()


list_testdic = []
for word in df_text: 
  try:    
    testdic = {}
    testdic['Casual'],testdic['Classic'],testdic['Modern'],testdic['Natural'] = 0,0,0,0
    temp = okt.nouns(word)
    for i in temp:
      if i in Casual:
        testdic['Casual'] += 1
      elif i in Classic:
        testdic['Classic'] += 1
      elif i in Modern:
        testdic['Modern'] += 1
      elif i in Natural:
        testdic['Natural'] += 1    
    list_testdic.append(testdic)
  except Exception as e:
    print(e)
    print(word)



# 뽑아낸 단어 수를 %로 나타내기
# temp_pd = pd.DataFrame(list_testdic)
import numpy as np
result_df_text = []
for dicdata in list_testdic:    
  datalist = list(dicdata.values())
  if(sum(datalist) == 0):
    result_df_text.append([0,0,0,0])
  else:
    result_df_text.append([
                    datalist[0] /  sum(datalist)
                   ,datalist[1] /  sum(datalist)
                   ,datalist[2] /  sum(datalist)
                   ,datalist[3] /  sum(datalist)
                   ])

result_df_text
result_df_text = list(map(lambda x : np.round(x, 3), result_df_text))        # result_df_text를 소숫점 3째자리에서 반올리해서 list 형태로 만들어라.

result_df_text

[array([0.   , 0.333, 0.667, 0.   ]),
 array([0, 0, 0, 0]),
 array([0, 0, 0, 0]),
 array([0., 0., 1., 0.]),
 array([0. , 0.5, 0. , 0.5]),
 array([0, 0, 0, 0]),
 array([0, 0, 0, 0]),
 array([0, 0, 0, 0]),
 array([0., 0., 1., 0.]),
 array([0., 0., 0., 1.]),
 array([0, 0, 0, 0]),
 array([0, 0, 0, 0]),
 array([0, 0, 0, 0]),
 array([0., 0., 0., 1.]),
 array([0, 0, 0, 0]),
 array([0, 0, 0, 0]),
 array([0, 0, 0, 0]),
 array([0, 0, 0, 0]),
 array([0.   , 0.333, 0.   , 0.667]),
 array([0, 0, 0, 0]),
 array([0, 0, 0, 0]),
 array([0., 0., 1., 0.]),
 array([0, 0, 0, 0]),
 array([0.   , 0.5  , 0.167, 0.333]),
 array([0, 0, 0, 0]),
 array([0., 0., 0., 1.]),
 array([0., 1., 0., 0.]),
 array([0., 0., 0., 1.]),
 array([0., 1., 0., 0.]),
 array([0., 0., 0., 1.]),
 array([0, 0, 0, 0]),
 array([0, 0, 0, 0]),
 array([0.   , 0.333, 0.5  , 0.167]),
 array([0. , 0. , 0.5, 0.5]),
 array([0, 0, 0, 0]),
 array([0, 0, 0, 0]),
 array([0, 0, 0, 0]),
 array([0, 0, 0, 0]),
 array([0, 0, 0, 0]),
 array([0., 0., 0., 1.

In [ ]:
# # review 단어 뽑아내기

# 각 스타일에 해당하는 10개 단어 (명사형으로 변형)
Casual = ['신선', '경쾌', '동적', '활동', '즐거운', '발랄', '재미', '친근', '따뜻', '산뜻']
Classic = ['고급', '우아', '고풍', '화려', '호화', '장식', '고전', '중후', '전통', '무거운']
Modern = ['단순', '현대', '깔끔', '깨끗', '간결', '간단', '심플', '기능', '도시', '합리', '절제', '세련']
Natural = ['자연', '여유', '온화', '부드러운', '포근', '전원', '편안', '솔직', '아늑', '자연의']

from konlpy.tag import Okt
okt = Okt()

list_testdic = []
for word in df_review:               ##
  testdic = {}
  testdic['Casual'],testdic['Classic'],testdic['Modern'],testdic['Natural'] = 0,0,0,0
  temp = okt.nouns(word)
  for i in temp:
    if i in Casual:
      testdic['Casual'] += 1
    elif i in Classic:
      testdic['Classic'] += 1
    elif i in Modern:
      testdic['Modern'] += 1
    elif i in Natural:
      testdic['Natural'] += 1    
  list_testdic.append(testdic)


# 뽑아낸 단어 수를 %로 나타내기
# temp_pd = pd.DataFrame(list_testdic)
import numpy as np
result_df_review = []
for dicdata in list_testdic:    
  datalist = list(dicdata.values())
  if(sum(datalist) == 0):
    result_df_review.append([0,0,0,0])
  else:
    result_df_review.append([
                    datalist[0] /  sum(datalist)
                   ,datalist[1] /  sum(datalist)
                   ,datalist[2] /  sum(datalist)
                   ,datalist[3] /  sum(datalist)
                   ])
    
result_df_review
result_df_review = list(map(lambda x : np.round(x, 3), result_df_review))

result_df_review

[array([0.  , 0.5 , 0.25, 0.25]),
 array([0., 0., 1., 0.]),
 array([0.4, 0.2, 0.2, 0.2]),
 array([0. , 0.2, 0.7, 0.1]),
 array([0.  , 0.25, 0.25, 0.5 ]),
 array([0, 0, 0, 0]),
 array([0.   , 0.   , 0.333, 0.667]),
 array([0.071, 0.214, 0.429, 0.286]),
 array([0.333, 0.   , 0.   , 0.667]),
 array([0.   , 0.75 , 0.182, 0.068]),
 array([0.   , 0.333, 0.   , 0.667]),
 array([0, 0, 0, 0]),
 array([0. , 0. , 0.5, 0.5]),
 array([0.2, 0.2, 0.2, 0.4]),
 array([0.053, 0.211, 0.368, 0.368]),
 array([0.294, 0.588, 0.118, 0.   ]),
 array([0.   , 0.   , 0.111, 0.889]),
 array([0. , 0. , 0.5, 0.5]),
 array([0.   , 0.333, 0.667, 0.   ]),
 array([0.   , 0.   , 0.667, 0.333]),
 array([0.2, 0. , 0.4, 0.4]),
 array([0., 0., 0., 1.]),
 array([0. , 0. , 0.2, 0.8]),
 array([0., 0., 0., 1.]),
 array([0.2, 0. , 0.2, 0.6]),
 array([0.333, 0.   , 0.333, 0.333]),
 array([0.   , 0.667, 0.333, 0.   ]),
 array([0., 0., 1., 0.]),
 array([0.4, 0. , 0.2, 0.4]),
 array([0.   , 0.308, 0.308, 0.385]),
 array([0.15, 0.1 , 

In [ ]:
# 소개글 모두 0인 것              >> 1221
count = 0
for i in result_df_text:
  if(sum(i)==0):
    count += 1
count


# # 리뷰 모두 0인 것     >> 124
# count = 0
# for i in result_df_review:
#   if(sum(i)==0):
#     count += 1
# count

1221

In [ ]:
type(result_df_review)
# , result_df_review, df_image

list

In [ ]:
a = pd.DataFrame(df_image)             # 이미지데이터
b = pd.DataFrame(result_df_review)*100     # 리뷰데이터_counting 한 것
c = pd.DataFrame(result_df_text)*100       # 텍스트데이터_counting 한 것
e = pd.DataFrame(df_cont)              # 통제변수들데이터
d = pd.DataFrame(df_score)             # 점수(종속변수)
d = d.reset_index(drop=True)

In [ ]:
df4 = pd.concat([a,b,c,e,d], axis=1)
df4

,0,1,2,3,0,1,2,3,0,1,2,3,가격,최대_인원수,침실개수,침대개수,욕실개수,평점
0,28.3,16.3,21.0,34.6,0.0,50.0,25.0,25.0,0.0,33.3,66.7,0.0,98940,4,1.0,1.0,1.0,4.67
1,47.4,7.8,24.7,20.2,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,59341,4,1.0,1.0,1.0,4.89
2,9.6,33.2,22.2,34.8,40.0,20.0,20.0,20.0,0.0,0.0,0.0,0.0,219562,5,2.0,1.0,1.0,4.99
3,34.2,5.1,52.2,8.7,0.0,20.0,70.0,10.0,0.0,0.0,100.0,0.0,84447,2,1.0,1.0,1.0,4.73
4,2.2,21.3,30.6,46.0,0.0,25.0,25.0,50.0,0.0,50.0,0.0,50.0,64000,2,1.0,1.0,1.0,4.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1725,30.1,21.4,39.7,8.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52951,2,0.0,1.0,1.0,4.65
1726,23.1,9.9,59.8,7.2,14.3,0.0,71.4,14.3,0.0,0.0,100.0,0.0,55918,4,1.0,2.0,1.0,4.92
1727,15.6,15.0,42.7,26.7,9.1,9.1,81.8,0.0,0.0,0.0,0.0,0.0,74177,2,1.0,1.0,1.0,4.83
1728,47.4,15.6,6.4,30.7,66.7,0.0,0.0,33.3,0.0,0.0,0.0,0.0,77030,4,1.0,1.0,1.0,4.95


In [ ]:
df4.columns = (['image_Casual', 'image_Classic', 'image_Modern', 'image_Natural','review_Casual', 'review_Classic', 'review_Modern', 'review_Natural', 'text_Casual', 'text_Classic', 'text_Modern', 'text_Natural', '가격',	'최대_인원수'	, '침실개수',	'침대개수',	'욕실개수',	'score'])

In [ ]:
df4

,image_Casual,image_Classic,image_Modern,image_Natural,review_Casual,review_Classic,review_Modern,review_Natural,text_Casual,text_Classic,text_Modern,text_Natural,가격,최대_인원수,침실개수,침대개수,욕실개수,score
0,28.3,16.3,21.0,34.6,0.0,50.0,25.0,25.0,0.0,33.3,66.7,0.0,98940,4,1.0,1.0,1.0,4.67
1,47.4,7.8,24.7,20.2,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,59341,4,1.0,1.0,1.0,4.89
2,9.6,33.2,22.2,34.8,40.0,20.0,20.0,20.0,0.0,0.0,0.0,0.0,219562,5,2.0,1.0,1.0,4.99
3,34.2,5.1,52.2,8.7,0.0,20.0,70.0,10.0,0.0,0.0,100.0,0.0,84447,2,1.0,1.0,1.0,4.73
4,2.2,21.3,30.6,46.0,0.0,25.0,25.0,50.0,0.0,50.0,0.0,50.0,64000,2,1.0,1.0,1.0,4.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1725,30.1,21.4,39.7,8.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52951,2,0.0,1.0,1.0,4.65
1726,23.1,9.9,59.8,7.2,14.3,0.0,71.4,14.3,0.0,0.0,100.0,0.0,55918,4,1.0,2.0,1.0,4.92
1727,15.6,15.0,42.7,26.7,9.1,9.1,81.8,0.0,0.0,0.0,0.0,0.0,74177,2,1.0,1.0,1.0,4.83
1728,47.4,15.6,6.4,30.7,66.7,0.0,0.0,33.3,0.0,0.0,0.0,0.0,77030,4,1.0,1.0,1.0,4.95


In [ ]:
df4.to_csv('predicted_values_1.csv')

In [ ]:
## 방법1) - 회귀분석 통계


# 필요한 요소들 import
import scipy
from scipy import stats
from sklearn.decomposition import PCA


# 전처리 (차수 or 차원 맞지 않을 때, 이용할 수 있는 방법)
pca1 = PCA(n_components=1)
x_pca = pca1.fit_transform(x)     # PCA를 이용한 차원축소(40,12) --> (40,1)로 축소시켜줌
x_pca = np.squeeze(x_pca)         # squeeze를 이용한 차원축소 (40,1)  --> (40,)로 축소시켜줌.
# x_pca.shape
y_pca = pca1.fit_transform(y)     # PCA를 이용한 차원축소(40,12) --> (40,1)로 축소시켜줌
y_pca = np.squeeze(y_pca)         # squeeze를 이용한 차원축소 (40,1)  --> (40,)로 축소시켜줌.
# x_pca.shape

# linregress로 통계 분석
# scipy.stats.linregress(x,y)
slope, intercept, rvalue, p_value, stderr = stats.linregress(x_pca,y_pca)    # slope_기울기, intercept_절편, rvalue_상관계수, p_value, stderr_에러표준편차


# 상관계수 출력
print('rvalue : ', rvalue)

# p-value
print('p_value : ', p_value) #0.05보다 적으면 유의미하다라고 판정
if p_value < 0.05:
  print('유의미하다')
else:
  print('유의미하지 않다')

# 에러 표준편차 출력
print('stderr : ',stderr)


# # p-value값 보기 위해서, e 대신 소숫점 8자리 수로 표현하는 함수
# x = 9.238421943157891e-05
# print(format(x,'.8f'))

In [ ]:
## 방법2) - 통계 분석
import statsmodels.api as sm

model = sm.OLS(y, x)
result = model.fit()

print(result.summary(xname = ['image_Casual', 'image_Classic', 'image_Modern', 'image_Natural','review_Casual', 'review_Classic', 'review_Modern', 'review_Natural', 'text_Casual', 'text_Classic', 'text_Modern', 'text_Natural']))

In [ ]:
## 방법3) - 통계 분석

from statsmodels.formula.api import ols
model = ols(formula = "y~x", data = df).fit()    # formula >> y값 ~ 종속변수 값 주고, data에는 dataFrame형식의 x_train값을 주면 됨.
model.summary()


# # 결과해석
#    - Dep. Variable: Dependent variable, 즉 종속변수를 의미한다.
#    - Model: 모델링 방법을 뜻하고, OLS는  Ordinary Least Squares의 약자이다.
#       ** 참고로 OLS란, 최소제곱법, 또는 최소자승법, 최소제곱근사법, 최소자승근사법(method of least squares, least squares approximation)이라고도 하며, 어떤 계의 해방정식을 근사적으로 구하는 방법으로서 근사적으로 구하려는 해와 실제 해의 오차의 제곱의 합이 최소가 되는 해를 구하는 방법이다. [출처: 위키백과]
#    - No. Observations: Number of observations, 관찰표본 수, 즉 총 표본 수를 뜻한다. 
#    - Df Residuals: DF는 Degree of Freedom으로 자유도를 뜻하는데, DF Residuals는 전체 표본 수에서 측정되는 변수들(종속변수 및 독립변수)의 개수를 빼서 구한다. 이번 회귀분석에서 DF Residuals를 구하자면,  전체 표본 수 50개 - 종속변수1개 - 독립변수1개 = 48로 구할 수 있다.
#    - Df Model: 독립변수의 개수이다.
#    - R squared: R의 제곱이라는 뜻이고 결정계수를 의미한다. 전체 데이터 중 해당 회귀모델이 설명할 수 있는 데이터의 비율, 회귀식의 설명력을 나타낸다. SSTr/SST이나 상관계수 R을 제곱해서 구할 수 있다. (상세 내용은 본 포스팅 #4.에서 추가설명 한다)
#    - F-statistics: F통계량을 뜻한다. F통계량은 MSR/MSE로 구할 수 있다. (상세 내용은 본 포스팅 #5. 에서 추가설명 한다)
#    - Prob: F통계량에 해당하는 P-value를 의미한다. P는 Probability의 첫글자이다.
#   [회귀계수]: 다음 두 값은 해당 모델의 회귀계수들이다.
#    - Intercept coef: Intercept coefficient는 회귀식의 절편 값을 의미한다.
#    - speed coef: 독립변수 coefficient는 독립변수 "speed"의 회귀계수를 의미하며, 회귀식에서 기울기를 의미한다. 
#      --> 위의 회귀계수들을 활용해 다음과 같이 회귀식을 수기로 작성할 수 있다.
#      --> y = 3.9324X -17.5791

In [ ]:
### 회귀분석 통계값 구하기
import statsmodels.api as sm

model = sm.OLS(y_1, x_1)
result = model.fit()

print(result.summary())

                                 OLS Regression Results                                
Dep. Variable:                  score   R-squared (uncentered):                   0.999
Model:                            OLS   Adj. R-squared (uncentered):              0.999
Method:                 Least Squares   F-statistic:                          1.662e+05
Date:                Mon, 21 Nov 2022   Prob (F-statistic):                        0.00
Time:                        11:37:54   Log-Likelihood:                          1228.7
No. Observations:                1730   AIC:                                     -2423.
Df Residuals:                    1713   BIC:                                     -2331.
Df Model:                          17                                                  
Covariance Type:            nonrobust                                                  
                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------